#### PyTorch CNN MNIST

In [1]:
import torch
import numpy as np

import idx2numpy
import matplotlib.cm as cm
import matplotlib.pyplot as plt

PATH = 'C:/Projects/keras_talk/keras/intern/0709/MNIST/'
EPOCHS = 10
BATCH_SIZE = 100
D=(60000//BATCH_SIZE) // 20

def one_hot_encoding( y, L ):
    Encoded_y = []
    N = len(y)
    for number in range(N):
        encoding = [0]*L
        np.asarray(encoding)
        encoding[ y[number] ] = 1
        Encoded_y.append(encoding)
    
    return np.asarray(Encoded_y)


#### data

In [2]:
x_test_path = 't10k-images.idx3-ubyte'
y_test_path = 't10k-labels.idx1-ubyte'
x_train_path = 'train-images.idx3-ubyte'
y_train_path = 'train-labels.idx1-ubyte'


x_test = idx2numpy.convert_from_file(PATH+x_test_path)
y_test = idx2numpy.convert_from_file(PATH+y_test_path)
x_train = idx2numpy.convert_from_file(PATH+x_train_path)
y_train = idx2numpy.convert_from_file(PATH+y_train_path)


x_test  = x_test.reshape ( len(x_test),  1, 28, 28).astype('float32') / 255.
x_train = x_train.reshape( len(x_train), 1, 28, 28).astype('float32') / 255.        
#y_test  = one_hot_encoding(y_test,10)
#y_train = one_hot_encoding(y_train,10)


#### Data_loader 구성

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


x_test  = torch.from_numpy(x_test ).float().to('cpu')
x_train = torch.from_numpy(x_train).float().to('cpu')
y_test  = torch.from_numpy(y_test ).long().to('cpu')
y_train = torch.from_numpy(y_train).long().to('cpu')


dataset      = TensorDataset(x_train, y_train)
dataset_test = TensorDataset(x_test, y_test)

dataloader      = DataLoader( dataset, batch_size = BATCH_SIZE
                        ,shuffle= True, drop_last = True ) 
dataloader_test = DataLoader( dataset_test, batch_size = BATCH_SIZE
                        ,shuffle= True, drop_last = True )

..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


#### pytorch model 구성

In [4]:
import torch.nn
import torch.nn.functional
import torch.optim

class CNN( torch.nn.Module ):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1  = torch.nn.Conv2d( in_channels =  1, out_channels = 16, kernel_size = 3 )
        self.Crelu1 = torch.nn.ReLU()
        self.conv2  = torch.nn.Conv2d( in_channels = 16, out_channels = 32, kernel_size = 3 )
        self.Crelu2 = torch.nn.ReLU()
        self.maxpl  = torch.nn.MaxPool2d( kernel_size=2 )
        #flatten()
        self.dropout= torch.nn.Dropout()
        self.lay1   = torch.nn.Linear( 12*12*32, 32 )
        self.Lrelu1 = torch.nn.ReLU()
        self.lay2   = torch.nn.Linear( 32, 10 )
        
        
    def forward(self, x):
        output = self.conv1(x)
        output = self.Crelu1(output)
        output = self.conv2(output)
        output = self.Crelu2(output)
        output = self.maxpl(output)
        output = output.view(-1,12*12*32)
        output = self.dropout(output)
        output = self.lay1(output)
        output = self.Lrelu1(output)
        output = self.lay2(output)
        output = torch.nn.functional.softmax(output, dim=1)
        
        return output
        


#### model 학습

In [5]:

torch.manual_seed(1)

model = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


model.train()
for epoch in range(EPOCHS):
    train_loss = 0.0
    
    print('epoch'+str(epoch+1)+':  [', end='')
    for i,(data, label) in enumerate(dataloader):

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        total = label.size(0)
        preds = torch.max(output.data,1)[1]
        correct = (preds == label).sum().item()
        
        
  
        #progress bar
        if D == i:
            print('%', end='')
            D += D
            
    print('%]')
    D = (60000//BATCH_SIZE)//20        
        
    #summary 
    print('    loss: ', loss.item(), end='\t')
    print('acc:  ', 100.*correct/total)




epoch1:  [%%%%%%]
    loss:  1.613397240638733	acc:   85.0
epoch2:  [%%%%%%]
    loss:  1.6085530519485474	acc:   87.0
epoch3:  [%%%%%%]
    loss:  1.5615794658660889	acc:   91.0
epoch4:  [%%%%%%]
    loss:  1.615219235420227	acc:   85.0
epoch5:  [%%%%%%]
    loss:  1.6781240701675415	acc:   79.0
epoch6:  [%%%%%%]
    loss:  1.6095837354660034	acc:   86.0
epoch7:  [%%%%%%]
    loss:  1.609041690826416	acc:   85.0
epoch8:  [%%%%%%]
    loss:  1.5835572481155396	acc:   88.0
epoch9:  [%%%%%%]
    loss:  1.5934165716171265	acc:   87.0
epoch10:  [%%%%%%]
    loss:  1.553545355796814	acc:   91.0


#### 학습 결과

In [6]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for data, label in dataloader_test:
        output = model(data)
        preds  = torch.max(output.data, 1)[1]
        total   += len(label)
        correct += (preds==label).sum().item()
    print('Test Accuracy: ', 100.*correct/total)

Test Accuracy:  87.0
